In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
from torch.utils.data import DataLoader
from torch import optim
import numpy as np
import argparse

import config
from ctpn_model import CTPN_Model, RPN_CLS_Loss, RPN_REGR_Loss
from data.dataset import ICDARDataset

# dataset_download:https://rrc.cvc.uab.es/?ch=8&com=downloads
random_seed = 2020
torch.random.manual_seed(random_seed)
np.random.seed(random_seed)

epochs = 30
lr = 1e-3
resume_epoch = 0


def save_checkpoint(state, epoch, loss_cls, loss_regr, loss, ext='pth'):
    check_path = os.path.join(config.checkpoints_dir,
                              f'v3_ctpn_ep{epoch:02d}_'
                              f'{loss_cls:.4f}_{loss_regr:.4f}_{loss:.4f}.{ext}')

    try:
        torch.save(state, check_path)
    except BaseException as e:
        print(e)
        print('fail to save to {}'.format(check_path))
    print('saving to {}'.format(check_path))

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
checkpoints_weight = config.pretrained_weights
print('exist pretrained ',os.path.exists(checkpoints_weight))
if os.path.exists(checkpoints_weight):
    pretrained = False
    print(pretrained)

exist pretrained  False


In [3]:
dataset = ICDARDataset(config.icdar19_mlt_img_dir, config.icdar19_mlt_gt_dir)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=config.num_workers)
model = CTPN_Model()
model.to(device)

CTPN_Model(
  (base_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1

In [5]:
if os.path.exists(checkpoints_weight):
    print('using pretrained weight: {}'.format(checkpoints_weight))
    cc = torch.load(checkpoints_weight, map_location=device)
    model.load_state_dict(cc['model_state_dict'])
    resume_epoch = cc['epoch']
else:
    model.apply(weights_init)

params_to_uodate = model.parameters()
optimizer = optim.SGD(params_to_uodate, lr=lr, momentum=0.9)

critetion_cls = RPN_CLS_Loss(device)
critetion_regr = RPN_REGR_Loss(device)

best_loss_cls = 100
best_loss_regr = 100
best_loss = 100
best_model = None
epochs += resume_epoch
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

for epoch in range(resume_epoch+1, epochs):
    torch.cuda.empty_cache()
    print(f'Epoch {epoch}/{epochs}')
    print('#'*50)
    epoch_size = len(dataset) // 1
    model.train()
    epoch_loss_cls = 0
    epoch_loss_regr = 0
    epoch_loss = 0
    scheduler.step(epoch)

    for batch_i, (imgs, clss, regrs) in enumerate(dataloader):
        torch.cuda.empty_cache()
        # print(imgs.shape)
        imgs = imgs.to(device)
        clss = clss.to(device)
        regrs = regrs.to(device)

        optimizer.zero_grad()

        out_cls, out_regr = model(imgs)
        loss_cls = critetion_cls(out_cls, clss)
        loss_regr = critetion_regr(out_regr, regrs)

        loss = loss_cls + loss_regr  # total loss
        loss.backward()
        optimizer.step()

        epoch_loss_cls += loss_cls.item()
        epoch_loss_regr += loss_regr.item()
        epoch_loss += loss.item()
        mmp = batch_i+1
        
        if batch_i%100 == 0:
            print(f'Ep:{epoch}/{epochs-1}--'
              f'Batch:{batch_i}/{epoch_size}\n'
              f'batch: loss_cls:{loss_cls.item():.4f}--loss_regr:{loss_regr.item():.4f}--loss:{loss.item():.4f}\n'
              f'Epoch: loss_cls:{epoch_loss_cls/mmp:.4f}--loss_regr:{epoch_loss_regr/mmp:.4f}--'
              f'loss:{epoch_loss/mmp:.4f}\n')

    epoch_loss_cls /= epoch_size
    epoch_loss_regr /= epoch_size
    epoch_loss /= epoch_size
    print(f'Epoch:{epoch}--{epoch_loss_cls:.4f}--{epoch_loss_regr:.4f}--{epoch_loss:.4f}')
    if best_loss_cls > epoch_loss_cls or best_loss_regr > epoch_loss_regr or best_loss > epoch_loss:
        best_loss = epoch_loss
        best_loss_regr = epoch_loss_regr
        best_loss_cls = epoch_loss_cls
        best_model = model
        save_checkpoint({'model_state_dict': best_model.state_dict(),
                         'epoch': epoch},
                        epoch,
                        best_loss_cls,
                        best_loss_regr,
                        best_loss)

if torch.cuda.is_available():
    torch.cuda.empty_cache()

Epoch 1/30
##################################################
Ep:1/29--Batch:0/3832
batch: loss_cls:0.7080--loss_regr:0.1501--loss:0.8580
Epoch: loss_cls:0.7080--loss_regr:0.1501--loss:0.8580

Ep:1/29--Batch:100/3832
batch: loss_cls:0.6947--loss_regr:0.1345--loss:0.8292
Epoch: loss_cls:0.6958--loss_regr:0.2062--loss:0.9020

Ep:1/29--Batch:200/3832
batch: loss_cls:0.6944--loss_regr:0.2911--loss:0.9855
Epoch: loss_cls:0.6925--loss_regr:0.2059--loss:0.8985

Ep:1/29--Batch:300/3832
batch: loss_cls:0.6951--loss_regr:0.1475--loss:0.8426
Epoch: loss_cls:0.6906--loss_regr:0.2145--loss:0.9051

Ep:1/29--Batch:400/3832
batch: loss_cls:0.6960--loss_regr:0.1753--loss:0.8713
Epoch: loss_cls:0.6890--loss_regr:0.2118--loss:0.9008

Ep:1/29--Batch:500/3832
batch: loss_cls:0.6966--loss_regr:0.1386--loss:0.8352
Epoch: loss_cls:0.6877--loss_regr:0.2082--loss:0.8960

Ep:1/29--Batch:600/3832
batch: loss_cls:0.6893--loss_regr:0.2495--loss:0.9388
Epoch: loss_cls:0.6858--loss_regr:0.2190--loss:0.9048

Ep:1/29--

Ep:2/29--Batch:2100/3832
batch: loss_cls:0.6981--loss_regr:0.0927--loss:0.7908
Epoch: loss_cls:0.6742--loss_regr:0.1940--loss:0.8682

Ep:2/29--Batch:2200/3832
batch: loss_cls:0.6510--loss_regr:0.1999--loss:0.8509
Epoch: loss_cls:0.6740--loss_regr:0.1930--loss:0.8670

Ep:2/29--Batch:2300/3832
batch: loss_cls:0.7160--loss_regr:0.1458--loss:0.8618
Epoch: loss_cls:0.6739--loss_regr:0.1922--loss:0.8660

Ep:2/29--Batch:2400/3832
batch: loss_cls:0.5929--loss_regr:0.1628--loss:0.7557
Epoch: loss_cls:0.6740--loss_regr:0.1929--loss:0.8669

Ep:2/29--Batch:2500/3832
batch: loss_cls:0.7111--loss_regr:0.1213--loss:0.8324
Epoch: loss_cls:0.6739--loss_regr:0.1916--loss:0.8655

Ep:2/29--Batch:2600/3832
batch: loss_cls:0.5349--loss_regr:2.2870--loss:2.8219
Epoch: loss_cls:0.6736--loss_regr:0.1914--loss:0.8650

Ep:2/29--Batch:2700/3832
batch: loss_cls:0.7169--loss_regr:0.2294--loss:0.9464
Epoch: loss_cls:0.6738--loss_regr:0.1908--loss:0.8646

Ep:2/29--Batch:2800/3832
batch: loss_cls:0.6835--loss_regr:0.1

Ep:4/29--Batch:200/3832
batch: loss_cls:0.7421--loss_regr:0.3954--loss:1.1375
Epoch: loss_cls:0.6565--loss_regr:0.1584--loss:0.8149

Ep:4/29--Batch:300/3832
batch: loss_cls:0.7325--loss_regr:0.1093--loss:0.8418
Epoch: loss_cls:0.6581--loss_regr:0.1685--loss:0.8266

Ep:4/29--Batch:400/3832
batch: loss_cls:0.4871--loss_regr:0.0833--loss:0.5703
Epoch: loss_cls:0.6552--loss_regr:0.1642--loss:0.8194

Ep:4/29--Batch:500/3832
batch: loss_cls:0.5570--loss_regr:0.0552--loss:0.6122
Epoch: loss_cls:0.6485--loss_regr:0.1611--loss:0.8096

Ep:4/29--Batch:600/3832
batch: loss_cls:0.7223--loss_regr:0.2678--loss:0.9901
Epoch: loss_cls:0.6474--loss_regr:0.1721--loss:0.8195

Ep:4/29--Batch:700/3832
batch: loss_cls:0.7337--loss_regr:0.0919--loss:0.8255
Epoch: loss_cls:0.6487--loss_regr:0.1706--loss:0.8193

Ep:4/29--Batch:800/3832
batch: loss_cls:0.7336--loss_regr:0.0821--loss:0.8157
Epoch: loss_cls:0.6486--loss_regr:0.1696--loss:0.8182

Ep:4/29--Batch:900/3832
batch: loss_cls:0.6960--loss_regr:0.1536--los

Ep:5/29--Batch:2400/3832
batch: loss_cls:0.5815--loss_regr:0.1172--loss:0.6986
Epoch: loss_cls:0.6212--loss_regr:0.1631--loss:0.7843

Ep:5/29--Batch:2500/3832
batch: loss_cls:0.6430--loss_regr:0.1169--loss:0.7599
Epoch: loss_cls:0.6210--loss_regr:0.1620--loss:0.7831

Ep:5/29--Batch:2600/3832
batch: loss_cls:0.4403--loss_regr:2.2695--loss:2.7097
Epoch: loss_cls:0.6210--loss_regr:0.1618--loss:0.7828

Ep:5/29--Batch:2700/3832
batch: loss_cls:0.6901--loss_regr:0.2611--loss:0.9512
Epoch: loss_cls:0.6209--loss_regr:0.1613--loss:0.7822

Ep:5/29--Batch:2800/3832
batch: loss_cls:0.4279--loss_regr:0.0826--loss:0.5104
Epoch: loss_cls:0.6211--loss_regr:0.1620--loss:0.7831

Ep:5/29--Batch:2900/3832
batch: loss_cls:0.6501--loss_regr:0.1025--loss:0.7526
Epoch: loss_cls:0.6211--loss_regr:0.1615--loss:0.7826

Ep:5/29--Batch:3000/3832
batch: loss_cls:0.5483--loss_regr:0.1474--loss:0.6957
Epoch: loss_cls:0.6200--loss_regr:0.1610--loss:0.7810

Ep:5/29--Batch:3100/3832
batch: loss_cls:0.6009--loss_regr:0.1

Ep:7/29--Batch:500/3832
batch: loss_cls:0.4431--loss_regr:0.0611--loss:0.5043
Epoch: loss_cls:0.5905--loss_regr:0.1465--loss:0.7370

Ep:7/29--Batch:600/3832
batch: loss_cls:0.7830--loss_regr:0.2903--loss:1.0733
Epoch: loss_cls:0.5851--loss_regr:0.1566--loss:0.7417

Ep:7/29--Batch:700/3832
batch: loss_cls:0.7357--loss_regr:0.0869--loss:0.8225
Epoch: loss_cls:0.5911--loss_regr:0.1545--loss:0.7456

Ep:7/29--Batch:800/3832
batch: loss_cls:0.5599--loss_regr:0.0684--loss:0.6283
Epoch: loss_cls:0.5913--loss_regr:0.1532--loss:0.7445

Ep:7/29--Batch:900/3832
batch: loss_cls:0.6375--loss_regr:0.1343--loss:0.7718
Epoch: loss_cls:0.5914--loss_regr:0.1488--loss:0.7402

Ep:7/29--Batch:1000/3832
batch: loss_cls:0.4866--loss_regr:0.1687--loss:0.6553
Epoch: loss_cls:0.5904--loss_regr:0.1478--loss:0.7382

Ep:7/29--Batch:1100/3832
batch: loss_cls:0.7545--loss_regr:0.1723--loss:0.9268
Epoch: loss_cls:0.5960--loss_regr:0.1504--loss:0.7464

Ep:7/29--Batch:1200/3832
batch: loss_cls:0.7639--loss_regr:0.2242--

...

Ep:17/29--Batch:3500/3832
batch: loss_cls:0.1837--loss_regr:0.0270--loss:0.2107
Epoch: loss_cls:0.4617--loss_regr:0.1168--loss:0.5785

Ep:17/29--Batch:3600/3832
batch: loss_cls:0.2539--loss_regr:0.0542--loss:0.3082
Epoch: loss_cls:0.4614--loss_regr:0.1169--loss:0.5783

Ep:17/29--Batch:3700/3832
batch: loss_cls:0.1933--loss_regr:0.0358--loss:0.2291
Epoch: loss_cls:0.4608--loss_regr:0.1170--loss:0.5779

Ep:17/29--Batch:3800/3832
batch: loss_cls:0.5989--loss_regr:0.1137--loss:0.7126
Epoch: loss_cls:0.4614--loss_regr:0.1174--loss:0.5788

Epoch:17--0.4618--0.1173--0.5791
saving to ../../checkpoints/v3_ctpn_ep17_0.4618_0.1173_0.5791.pth
Epoch 18/30
##################################################
Ep:18/29--Batch:0/3832
batch: loss_cls:0.3664--loss_regr:0.0868--loss:0.4532
Epoch: loss_cls:0.3664--loss_regr:0.0868--loss:0.4532

Ep:18/29--Batch:100/3832
batch: loss_cls:0.6364--loss_regr:0.1833--loss:0.8197
Epoch: loss_cls:0.4665--loss_regr:0.1172--loss:0.5838

Ep:18/29--Batch:200/3832
batch: 

Ep:19/29--Batch:1600/3832
batch: loss_cls:0.2729--loss_regr:0.0555--loss:0.3284
Epoch: loss_cls:0.4548--loss_regr:0.1110--loss:0.5658

Ep:19/29--Batch:1700/3832
batch: loss_cls:0.6079--loss_regr:0.1493--loss:0.7573
Epoch: loss_cls:0.4540--loss_regr:0.1116--loss:0.5656

Ep:19/29--Batch:1800/3832
batch: loss_cls:0.2808--loss_regr:0.0562--loss:0.3370
Epoch: loss_cls:0.4517--loss_regr:0.1111--loss:0.5628

Ep:19/29--Batch:1900/3832
batch: loss_cls:0.2622--loss_regr:0.2260--loss:0.4882
Epoch: loss_cls:0.4524--loss_regr:0.1145--loss:0.5669

Ep:19/29--Batch:2000/3832
batch: loss_cls:0.3490--loss_regr:0.1176--loss:0.4666
Epoch: loss_cls:0.4509--loss_regr:0.1145--loss:0.5654

Ep:19/29--Batch:2100/3832
batch: loss_cls:0.3529--loss_regr:0.0365--loss:0.3894
Epoch: loss_cls:0.4523--loss_regr:0.1138--loss:0.5661

Ep:19/29--Batch:2200/3832
batch: loss_cls:0.5683--loss_regr:0.0723--loss:0.6405
Epoch: loss_cls:0.4509--loss_regr:0.1132--loss:0.5640

Ep:19/29--Batch:2300/3832
batch: loss_cls:0.6305--loss_

Ep:20/29--Batch:3700/3832
batch: loss_cls:0.2646--loss_regr:0.0577--loss:0.3224
Epoch: loss_cls:0.4438--loss_regr:0.1104--loss:0.5542

Ep:20/29--Batch:3800/3832
batch: loss_cls:0.5413--loss_regr:0.1105--loss:0.6517
Epoch: loss_cls:0.4441--loss_regr:0.1107--loss:0.5548

Epoch:20--0.4444--0.1107--0.5551
saving to ../../checkpoints/v3_ctpn_ep20_0.4444_0.1107_0.5551.pth
Epoch 21/30
##################################################
Ep:21/29--Batch:0/3832
batch: loss_cls:0.2765--loss_regr:0.0778--loss:0.3543
Epoch: loss_cls:0.2765--loss_regr:0.0778--loss:0.3543

Ep:21/29--Batch:100/3832
batch: loss_cls:0.6547--loss_regr:0.1912--loss:0.8459
Epoch: loss_cls:0.4621--loss_regr:0.1131--loss:0.5752

Ep:21/29--Batch:200/3832
batch: loss_cls:0.7900--loss_regr:0.2397--loss:1.0297
Epoch: loss_cls:0.4490--loss_regr:0.1078--loss:0.5568

Ep:21/29--Batch:300/3832
batch: loss_cls:0.7354--loss_regr:0.0581--loss:0.7935
Epoch: loss_cls:0.4514--loss_regr:0.1080--loss:0.5595

Ep:21/29--Batch:400/3832
batch: lo

Ep:22/29--Batch:1800/3832
batch: loss_cls:0.2165--loss_regr:0.0717--loss:0.2882
Epoch: loss_cls:0.4404--loss_regr:0.1074--loss:0.5478

Ep:22/29--Batch:1900/3832
batch: loss_cls:0.3080--loss_regr:0.2540--loss:0.5620
Epoch: loss_cls:0.4413--loss_regr:0.1106--loss:0.5519

Ep:22/29--Batch:2000/3832
batch: loss_cls:0.3613--loss_regr:0.1077--loss:0.4689
Epoch: loss_cls:0.4400--loss_regr:0.1107--loss:0.5507

Ep:22/29--Batch:2100/3832
batch: loss_cls:0.3160--loss_regr:0.0286--loss:0.3445
Epoch: loss_cls:0.4413--loss_regr:0.1100--loss:0.5513

Ep:22/29--Batch:2200/3832
batch: loss_cls:0.4113--loss_regr:0.1020--loss:0.5133
Epoch: loss_cls:0.4395--loss_regr:0.1095--loss:0.5490

Ep:22/29--Batch:2300/3832
batch: loss_cls:0.6225--loss_regr:0.1275--loss:0.7500
Epoch: loss_cls:0.4392--loss_regr:0.1091--loss:0.5482

Ep:22/29--Batch:2400/3832
batch: loss_cls:0.6324--loss_regr:0.0406--loss:0.6730
Epoch: loss_cls:0.4402--loss_regr:0.1099--loss:0.5500

Ep:22/29--Batch:2500/3832
batch: loss_cls:0.4129--loss_

KeyboardInterrupt: 

In [6]:
device

device(type='cuda', index=0)

In [11]:
loss

tensor(0.2913, device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
!fuser -v /dev/nvidia*

In [ ]:
!nvidia-smi  